In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker 
import pycountry_convert as pc
import folium
import branca
from datetime import datetime, timedelta,date
from scipy.interpolate import make_interp_spline, BSpline
import plotly.express as px
import json, requests
import calmap

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## Load data

In [2]:
# Retriving Dataset
df_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

# Depricated
# df_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')
df_covid19 = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv")
df_table = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_time.csv",parse_dates=['Last_Update'])

## Preprocess Data

In [3]:
df_covid19 = df_covid19.drop(["People_Tested","People_Hospitalized","UID","ISO3","Mortality_Rate"],axis =1)
df_covid19.head(2)

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate
0,Afghanistan,2020-12-05 23:26:05,33.93911,67.709953,47072.0,1864.0,37393.0,7815.0,120.919615
1,Albania,2020-12-05 23:26:05,41.15330,20.168300,42148.0,889.0,21286.0,19973.0,1464.591007


In [5]:
df_confirmed = df_confirmed.rename(columns={"Province/State":"state","Country/Region": "country"})
df_deaths = df_deaths.rename(columns={"Province/State":"state","Country/Region": "country"})
df_covid19 = df_covid19.rename(columns={"Country_Region": "country"})
df_covid19["Active"] = df_covid19["Confirmed"]-df_covid19["Recovered"]-df_covid19["Deaths"]
# df_recovered = df_recovered.rename(columns={"Province/State":"state","Country/Region": "country"})

In [7]:
df_table = df_table[df_table["continent"] != "Others"]

In [8]:
df_deaths[df_deaths["continent" ]== 'Others']

,state,country,continent,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20
34,NaN,Burma,Others,21.916200,95.956000,0,0,0,0,0,...,1810,1846,1865,1887,1918,1941,1972,1998,2028,2059
104,NaN,Diamond Princess,Others,0.000000,0.000000,0,0,0,0,0,...,13,13,13,13,13,13,13,13,13,13
141,NaN,Holy See,Others,41.902900,12.453400,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
158,NaN,Kosovo,Others,42.602636,20.902977,0,0,0,0,0,...,948,963,975,989,1005,1016,1026,1026,1026,1026
170,NaN,MS Zaandam,Others,0.000000,0.000000,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
241,NaN,Timor-Leste,Others,-8.874217,125.727539,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
266,NaN,West Bank and Gaza,Others,31.952200,35.233200,0,0,0,0,0,...,665,680,689,704,717,732,747,763,780,797
267,NaN,Western Sahara,Others,24.215500,-12.885800,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [9]:
df_confirmed = df_confirmed.replace(np.nan, '', regex=True)
df_deaths = df_deaths.replace(np.nan, '', regex=True)
# df_recovered = df_recovered.replace(np.nan, '', regex=True)
# df_active = df_active.replace(np.nan, '', regex=True)

In [11]:
df_countries_cases = df_covid19.copy().drop(['Lat','Long_','continent','Last_Update'],axis =1)
df_countries_cases.index = df_countries_cases["country"]
df_countries_cases = df_countries_cases.drop(['country'],axis=1)

df_continents_cases = df_covid19.copy().drop(['Lat','Long_','country','Last_Update'],axis =1)
df_continents_cases = df_continents_cases.groupby(["continent"]).sum()

df_countries_cases.fillna(0,inplace=True)
df_continents_cases.fillna(0,inplace=True)

In [12]:
df_t = pd.DataFrame(pd.to_numeric(df_countries_cases.sum()),dtype=np.float64).transpose()
df_t["Mortality Rate (per 100)"] = np.round(100*df_t["Deaths"]/df_t["Confirmed"],2)
df_t.style.background_gradient(cmap='Wistia',axis=1).format("{:.2f}").format("{:.0f}",subset=["Confirmed","Deaths","Recovered","Active"])

,Confirmed,Deaths,Recovered,Active,Incident_Rate,Mortality Rate (per 100)
0,66442818,1527209,42576816,21284699,239227.97,2.30


In [13]:
df_continents_cases["Mortality Rate (per 100)"] = np.round(100*df_continents_cases["Deaths"]/df_continents_cases["Confirmed"],2)
df_continents_cases.style.background_gradient(cmap='Blues',subset=["Confirmed"])\
                        .background_gradient(cmap='Reds',subset=["Deaths"])\
                        .background_gradient(cmap='Greens',subset=["Recovered"])\
                        .background_gradient(cmap='Purples',subset=["Active"])\
                        .background_gradient(cmap='Pastel1_r',subset=["Incident_Rate"])\
                        .background_gradient(cmap='YlOrBr',subset=["Mortality Rate (per 100)"])\
                        .format("{:.2f}")\
                        .format("{:.0f}",subset=["Confirmed","Deaths","Recovered","Active"])

,Confirmed,Deaths,Recovered,Active,Incident_Rate,Mortality Rate (per 100)
continent,,,,,,
Africa,2249651,53554,1917908,278189,12960.13,2.38
Asia,17199466,297123,15201936,1700407,50929.26,1.73
Australia,30771,942,28270,1559,175.10,3.06
Europe,18322208,420704,7726164,9121246,124202.28,2.30
North America,16923883,419393,7256419,9248071,22589.21,2.48
Others,237271,3996,176198,57077,7744.02,1.68
South America,11479568,331497,10269921,878150,20627.97,2.89


## Sort countries by Incident Rate

In [15]:
df_countries_cases["Mortality Rate (per 100)"] = np.round(100*df_countries_cases["Deaths"]/df_countries_cases["Confirmed"],2)
df_countries_cases.sort_values('Incident_Rate', ascending= False).style.background_gradient(cmap='Blues',subset=["Confirmed"])\
                        .background_gradient(cmap='Reds',subset=["Deaths"])\
                        .background_gradient(cmap='Greens',subset=["Recovered"])\
                        .background_gradient(cmap='Purples',subset=["Active"])\
                        .background_gradient(cmap='Pastel1_r',subset=["Incident_Rate"])\
                        .background_gradient(cmap='YlOrBr',subset=["Mortality Rate (per 100)"])\
                        .format("{:.2f}")\
                        .format("{:.0f}",subset=["Confirmed","Deaths","Recovered","Active"])

,Confirmed,Deaths,Recovered,Active,Incident_Rate,Mortality Rate (per 100)
country,,,,,,
Andorra,7005,78,6171,756,9066.20,1.11
Montenegro,37879,530,26809,10540,6031.09,1.40
Luxembourg,37017,345,28029,8643,5913.49,0.93
San Marino,1789,48,1412,329,5271.38,2.68
Bahrain,87732,341,85876,1515,5155.90,0.39
Czechia,544112,8782,474167,61163,5080.89,1.61
Belgium,587439,17142,0,0,5068.67,2.92
Qatar,139783,239,137060,2484,4851.79,0.17
Armenia,140959,2309,116066,22584,4756.93,1.64


## We focus on reletive values rather than absolute values.

## For Incident Rate (per 100,000):
### Top 25 countries:(those have highest incident rate)
Andorra  AD,Montenegro  ME(CS),Luxembourg  LU,San Marino  SM,Bahrain  BH,Czechia  CZ,Belgium  BE,Qatar  QA,Armenia  AM,USA  US,Slovenia  SI,Panama  PA,Switzerland  CH,Israel  IL,Georgia  GE,Liechtenstein  LI, Spain  ES,Croatia  HR,France  FR,Kuwait  KW,Austria  AT,Netherlands  NL,North Macedonia  MK,Argentina  AR,Portugal  PT,Brazil  BR,Peru  PE,Chile  CL,Moldova  MD.
### Bottom 25 countries:(those have lowest incident rate)
Vietnam  VN,Thailand  TH,Burundi  BI,China  CN,Niger  NE,Papua New Guinea  PG,Yemen  YE,Chad  TD,Democratic Republic of the Congo  CD,Burkina Faso  BF,Eritrea  ER,Mali  ML,Benin  BJ,Mongolia  MN,South Sudan  SS,Somalia  SO,Sierra Leone  SL,Malawi  MW,Liberia  LR,Nigeria  NG,Togo  TG,Mauritius  MU,New Zealand  NZ,Sudan  SD,Rwanda  RW.

## Sort countries by Mortality Rate

In [16]:
df_countries_cases.sort_values('Mortality Rate (per 100)', ascending= False).style.background_gradient(cmap='Blues',subset=["Confirmed"])\
                        .background_gradient(cmap='Reds',subset=["Deaths"])\
                        .background_gradient(cmap='Greens',subset=["Recovered"])\
                        .background_gradient(cmap='Purples',subset=["Active"])\
                        .background_gradient(cmap='Pastel1_r',subset=["Incident_Rate"])\
                        .background_gradient(cmap='YlOrBr',subset=["Mortality Rate (per 100)"])\
                        .format("{:.2f}")\
                        .format("{:.0f}",subset=["Confirmed","Deaths","Recovered","Active"])

,Confirmed,Deaths,Recovered,Active,Incident_Rate,Mortality Rate (per 100)
country,,,,,,
Yemen,2304,633,1547,124,7.72,27.47
MS Zaandam,9,2,7,0,0.00,22.22
Western Sahara,10,1,8,1,1.67,10.00
Mexico,1156770,108863,852719,195188,905.20,9.41
Ecuador,197391,13756,169804,13831,1118.80,6.97
Sudan,19196,1290,10942,6964,43.78,6.72
Bolivia,145325,8987,124390,11948,1244.96,6.18
Chad,1722,102,1561,59,10.48,5.92
Egypt,118014,6750,103324,7940,115.32,5.72


## For Mortality Rate (per 100):
### Top 25 countries:(those have highest mortality rate)
Yemen,Mexico,Ecuador,Sudan,Bolivia,Chad,Egypt,Syria,China,Liberia,Iran,Niger,Tanzania,Afghanistan,Peru,United Kingdom,Italy,Tunisia,Guatemala,Gambia,Australia,Mali,Bosnia and Herzegovina,Indonesia,Sierra Leone				
### Bottom 25 countries:(choose whose total cases are more than 500)
Eritrea,Mongolia,Singapore,Burundi,Qatar,Botswana,United Arab Emirates,Maldives,Bahrain,Cyprus,Monaco,Iceland,Sri Lanka,Malaysia,Guinea,Côte d'Ivoire,Kuwait,Ghana,Gabon,Nepal,Tajikistan,Lebanon,Mozambique,Belarus,Uzbekistan	

## I think "incident rate" is more relevant to how well each country try to stop the spread of COVID-19, and mortality rate is more relevant to the medical resource and healthcare system of one country. So I decide just to focus on "incident rate" to evaluate the performance.